In [140]:
import pandas as pd
import os
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix

from sklearn.preprocessing import label_binarize, StandardScaler, LabelEncoder, MinMaxScaler

from imblearn.over_sampling import SMOTE

import seaborn as sns
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

from joblib import dump

In [141]:
folder_path_machine = 'MachineLearningCVE'

In [142]:
data1 = pd.read_csv("MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")

data2 = pd.read_csv("MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")

data3 = pd.read_csv("MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv")

data4 = pd.read_csv("MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv")

data5 = pd.read_csv("MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")

data6 = pd.read_csv("MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")

data7 = pd.read_csv("MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv")

data8 = pd.read_csv("MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv")


In [143]:
data1

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225740,61374,61,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
225741,61378,72,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
225742,61375,75,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
225743,61323,48,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [144]:
dfs = []

In [145]:
data1['Label'].value_counts()

Label
DDoS      128027
BENIGN     97718
Name: count, dtype: int64

In [146]:
rows, cols = data1.shape
print(rows, cols)
sample_size = int(rows * 0.01)
sampled_df = data1.sample(n=sample_size, random_state=42)
dfs.append(sampled_df)
sampled_df['Label'].value_counts()

225745 79


Label
DDoS      1305
BENIGN     952
Name: count, dtype: int64

In [147]:
data2['Label'].value_counts()

Label
PortScan    158930
BENIGN      127537
Name: count, dtype: int64

In [148]:
rows, cols = data2.shape
print(rows, cols)
sample_size = int(rows * 0.01)
sampled_df = data2.sample(n=sample_size, random_state=42)
dfs.append(sampled_df)
sampled_df['Label'].value_counts()

286467 79


Label
PortScan    1633
BENIGN      1231
Name: count, dtype: int64

In [149]:
data3['Label'].value_counts()

Label
BENIGN    189067
Bot         1966
Name: count, dtype: int64

In [150]:
benign_sample = data3[data3['Label'] == 'BENIGN'].sample(frac=0.01, random_state=42)
bot_sample = data3[data3['Label'] == 'Bot'].sample(frac=0.2, random_state=42)

sampled_df = pd.concat([benign_sample, bot_sample])

dfs.append(sampled_df)
print(sampled_df['Label'].value_counts())

Label
BENIGN    1891
Bot        393
Name: count, dtype: int64


In [151]:
data4['Label'].value_counts() # Todo: Remove this data 4

Label
BENIGN    529918
Name: count, dtype: int64

In [152]:
data5['Label'].value_counts()

Label
BENIGN          288566
Infiltration        36
Name: count, dtype: int64

In [153]:
benign_sample = data5[data5['Label'] == 'BENIGN'].sample(frac=0.01, random_state=42)
bot_sample = data5[data5['Label'] == 'Infiltration'].sample(frac=1, random_state=42)

sampled_df = pd.concat([benign_sample, bot_sample])

dfs.append(sampled_df)
print(sampled_df['Label'].value_counts())

Label
BENIGN          2886
Infiltration      36
Name: count, dtype: int64


In [154]:
data6['Label'].value_counts()

Label
BENIGN                      168186
Web Attack Brute Force        1507
Web Attack XSS                 652
Web Attack Sql Injection        21
Name: count, dtype: int64

In [155]:
benign_sample = data6[data6['Label'] == 'BENIGN'].sample(frac=0.01, random_state=42)
Web_Attack_Brute_Force = data6[data6['Label'] == 'Web Attack Brute Force'].sample(frac=0.3, random_state=42)
web_attack_xss = data6[data6['Label'] == 'Web Attack XSS'].sample(frac=0.3, random_state=42)
web_attack_sql_injection = data6[data6['Label'] == 'Web Attack Sql Injection'].sample(frac=1, random_state=42)

sampled_df = pd.concat([benign_sample, Web_Attack_Brute_Force, web_attack_xss, web_attack_sql_injection])

dfs.append(sampled_df)
print(sampled_df['Label'].value_counts())

Label
BENIGN                      1682
Web Attack Brute Force       452
Web Attack XSS               196
Web Attack Sql Injection      21
Name: count, dtype: int64


In [156]:
data7['Label'].value_counts()

Label
BENIGN         432074
FTP-Patator      7938
SSH-Patator      5897
Name: count, dtype: int64

In [157]:
benign_sample = data7[data7['Label'] == 'BENIGN'].sample(frac=0.001, random_state=42)
ftp_patator = data7[data7['Label'] == 'FTP-Patator'].sample(frac=0.1, random_state=42)
ssh_patator = data7[data7['Label'] == 'SSH-Patator'].sample(frac=0.1, random_state=42)

sampled_df = pd.concat([benign_sample, ftp_patator, ssh_patator])

dfs.append(sampled_df)
print(sampled_df['Label'].value_counts())

Label
FTP-Patator    794
SSH-Patator    590
BENIGN         432
Name: count, dtype: int64


In [158]:
data8['Label'].value_counts()

Label
BENIGN              440031
DoS Hulk            231073
DoS GoldenEye        10293
DoS slowloris         5796
DoS Slowhttptest      5499
Heartbleed              11
Name: count, dtype: int64

In [159]:
benign_sample = data8[data8['Label'] == 'BENIGN'].sample(frac=0.001, random_state=42)
dos_hulk = data8[data8['Label'] == 'DoS Hulk'].sample(frac=0.01, random_state=42)
dos_goldeneye = data8[data8['Label'] == 'DoS GoldenEye'].sample(frac=0.1, random_state=42)
dos_slowloris = data8[data8['Label'] == 'DoS slowloris'].sample(frac=0.1, random_state=42)
dos_slowhttptest = data8[data8['Label'] == 'DoS Slowhttptest'].sample(frac=0.1, random_state=42)
heartbleed = data8[data8['Label'] == 'Heartbleed'].sample(frac=1, random_state=42)

sampled_df = pd.concat([benign_sample, dos_hulk, dos_goldeneye, dos_slowloris, dos_slowhttptest, heartbleed])

dfs.append(sampled_df)
print(sampled_df['Label'].value_counts())

Label
DoS Hulk            2311
DoS GoldenEye       1029
DoS slowloris        580
DoS Slowhttptest     550
BENIGN               440
Heartbleed            11
Name: count, dtype: int64


In [160]:
final_dataset = pd.concat(dfs, ignore_index=True)
final_dataset['Label'].value_counts()

Label
BENIGN                      9514
DoS Hulk                    2311
PortScan                    1633
DDoS                        1305
DoS GoldenEye               1029
FTP-Patator                  794
SSH-Patator                  590
DoS slowloris                580
DoS Slowhttptest             550
Web Attack Brute Force       452
Bot                          393
Web Attack XSS               196
Infiltration                  36
Web Attack Sql Injection      21
Heartbleed                    11
Name: count, dtype: int64

In [161]:
final_dataset

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,9392887,4,0,24,0,6,6,6.000000,0.000000,...,20,1.982000e+03,0.0000,1982,1982,9390905.0,0.0000,9390905,9390905,DDoS
1,64007,5438011,1,5,6,30,6,6,6.000000,0.000000,...,20,2.715100e+04,0.0000,27151,27151,5410860.0,0.0000,5410860,5410860,BENIGN
2,80,115615609,20,15,1728,3463,578,0,86.400000,211.017934,...,32,1.609163e+05,134508.6864,566475,120118,10100000.0,46752.4843,10100000,9988018,BENIGN
3,80,9827,3,5,26,11601,20,0,8.666667,10.263203,...,20,0.000000e+00,0.0000,0,0,0.0,0.0000,0,0,DDoS
4,53315,87583370,7,9,11607,62,4380,0,1658.142857,1762.272815,...,20,4.219842e+06,0.0000,4219842,4219842,82600000.0,0.0000,82600000,82600000,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19410,444,119299621,2805,2028,13712,7878627,5792,0,4.888414,110.120790,...,32,0.000000e+00,0.0000,0,0,0.0,0.0000,0,0,Heartbleed
19411,444,119297996,2782,2089,9368,7882432,1448,0,3.367362,30.505030,...,32,0.000000e+00,0.0000,0,0,0.0,0.0000,0,0,Heartbleed
19412,444,119257653,2802,2067,20858,7812389,5792,0,7.443969,126.045811,...,32,0.000000e+00,0.0000,0,0,0.0,0.0000,0,0,Heartbleed
19413,444,119260295,2791,2114,13712,7878088,5792,0,4.912934,110.396187,...,32,0.000000e+00,0.0000,0,0,0.0,0.0000,0,0,Heartbleed


In [ ]:
output_file = os.path.join("CIC-IDS-2017-compress", "cic-ids_dataset.csv")
final_dataset.to_csv(output_file, index=False)